卷积网络中的输入和层与传统神经网络有些区别，需要重新设计，训练模块基本一致

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from torchvision import datasets, transforms

**读取数据**
- 分别构建训练集和测试集（验证集）
- DataLoader 来迭代选取数据

In [2]:
# 定义超参数
input_size = 28
classes = 10
epochs = 3
batch_size = 64

# 定义数据预处理
transform = transforms.ToTensor()

# 训练集
train_dataset = datasets.MNIST(root="./data",        # 加载数据集
                              train=True,            # 决定加载的是数据集还是测试集
                              transform=transform,   # 数据预处理操作：转为tensor（张量）格式
                              download=True)         # 如果指定文件夹没数据，就从官网下载
# 测试集
valid_dataset = datasets.MNIST(root="./data",
                             train=False,
                             transform=transform)

# 构建 batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

**卷积网络构建**
- 一般卷积层，relu层，池化层可以写在一起
- 注意卷积最后结果还是一个特征图，需要把特征图转换成向量才能做分类或者回归任务

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(     # nn.Sequential() 能够把多个神经网络层封装成一个整体
            nn.Conv2d(                  # 卷积层，输入是（64， 1， 28， 28）
                in_channels=1,          # 灰度图
                out_channels=16,        # 要得到多少特征图
                kernel_size=5,          # 卷积核大小 5*5
                stride=1,               # 步长
                padding=2               # 边缘
            ),                          # 输出的特征图大小为（64， 16， 28， 28）
            nn.ReLU(),                  # 激活函数
            nn.MaxPool2d(kernel_size=2) # 进行池化操作，输出结果为（64， 16， 14， 14）
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(                  # 输入是（64，16，14，14）
                in_channels=16,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2
            ),                          # 输出是（64，32，14，14）
            nn.ReLU(),              
            nn.MaxPool2d(kernel_size=2) # 输出是（64，32，7，7）
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 32, 5, 1, 2), # 输出是（64，32，7，7）
            nn.ReLU(),                  
            nn.Conv2d(32, 64, 5, 1, 2), # 输出是（64，64，7，7）
            nn.ReLU(),
        )
        # 最终是要进行是分类，因此要将特征图转展平变成（64， 64 * 7 * 7）再传入全连接层计算分类结果
        # 输出是（64，10）
        self.out = nn.Linear(64 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)    # 展平操作，结果为(batch_size, 64 * 7 * 7)
        output = self.out(x)
        return output                # 结果矩阵，大小为64 * 10

**求准确率，准确率作为评估标准**

In [4]:
def seek_right_num(predictions, labels):
    pred_labels = torch.max(predictions.data, 1)[1]
    # 计算预测标签和真实标签之间匹配的数量
    # view.as() 方法尝试将 labels 张量调整为与 pred_labels 相同的形状
    right_num = pred_labels.eq(labels.data.view_as(pred_labels)).sum()
    return right_num, len(labels)

**训练网络模型**

In [8]:
# 网络（其实就是模型）
net = CNN()
# 损失函数
loss_func = torch.nn.CrossEntropyLoss()
# 优化器
adam_optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(epochs):
    # 保存训练集的准确度
    right_nums = []
    for batch_idx, (batch_x_train, batch_y_train) in enumerate(train_loader):
        # 训练模式
        net.train()
        # 预测值
        batch_pred = net(batch_x_train)
        # 计算该batch中准确的标签数量及其该batch数量
        right_num = seek_right_num(batch_pred, batch_y_train)    # 接收到的是一个元组
        right_nums.append(right_num)
        
        # 损失值
        loss = loss_func(batch_pred, batch_y_train)
        # 清空梯度
        adam_optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新参数
        adam_optimizer.step()

        # if batch_idx % 100 == 0:
        #     print(right_nums)
        #     net.eval()
        #     valid_right_nums = []

        #     for (batch_x_valid, batch_y_valid) in valid_loader:
        #         pred_valid = net(batch_x_valid)
        #         right_num = seek_right_num(pred_valid, batch_y_valid)
        #         valid_right_nums.append(right_num)

        #     # 准确率计算
        #     train_accuracy = (sum([tup[0] for tup in right_nums]), sum([tup[1] for tup in right_nums]))
        #     valid_accuracy = (sum([tup[0] for tup in valid_right_nums]), sum([tup[1] for tup in valid_right_nums]))

        #     print("当前epoch: {} [{} / {} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}%".format(
        #         epoch, batch_idx * batch_size, len(train_loader.dataset),
        #         100. * batch_idx / len(train_loader),
        #         loss.data,
        #         100. * train_accuracy[0].numpy() / train_accuracy[1],
        #         100. * valid_accuracy[0].numpy() / valid_accuracy[1]
        #     ))

[(tensor(5), 64)]


KeyboardInterrupt: 